# MUST RUN AT THE START OF EVERYTHING

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib inline

from snorkel import SnorkelSession
session = SnorkelSession()

In [ ]:
from snorkel.models import candidate_subclass

DiseaseGene = candidate_subclass('DiseaseGene', ['Disease', 'Gene'])

# Look at potential Candidates

Use this to look at loaded candidates from a given set. The constants represent the index to retrieve the training set, development set and testing set.

In [ ]:
from snorkel.viewer import SentenceNgramViewer

TRAIN = 0
DEV = 1
TEST = 2

candidates = session.query(DiseaseGene).filter(DiseaseGene.split==TRAIN).all()
sv = SentenceNgramViewer(candidates, session)

In [ ]:
sv

# Label Functions

Here is the fundamental part of the project. Below are the label functions that are used to give a candidate a label of 1,0 or -1 which corresponds to correct relation, not sure and incorrection relation. The goal here is to develop functions that can label as many candidates as possible.

In [ ]:
import re
from snorkel.lf_helpers import (
    get_left_tokens,
    get_right_tokens, 
    get_between_tokens,
    get_tagged_text,
    get_text_between,
    rule_regex_search_tagged_text,
    rule_regex_search_btw_AB,
    rule_regex_search_btw_BA,
    rule_regex_search_before_A,
    rule_regex_search_before_B,
)

In [ ]:
#insert label functions here!!!
#For my hackish debugger purposes only
def LF_debugger(c):
    print c
    print
    print vars(c)
    print
    print vars(c[0])
    print
    print c[0]
    print 
    print c[0].get_span()
    print
    print get_tagged_text(c)
    print
    print get_text_between(c)
    print
    print get_between_tokens(c)
    print
    print get_left_tokens(c)
    print
    print get_right_tokens(c)
    print
    sys.exit(1)
    return 1 if c else 0

def LF_between_tag(c):
    m = re.search("associated with|Disruption",get_text_between(c))
    return 1 if m else 0

def LF_mutation(c):
    m = re.search("mutation", ",".join(get_left_tokens(c)))
    n = re.search("mutation", ",".join(get_right_tokens(c)))
    return 1 if m or n else 0

def LF_check_disease_tag(c):
    disease_name = c[0].get_span()
    if "syndrome" in disease_name:
        if "epilepsy" in disease_name.replace("syndrome",""):
            return 1
        else:
            return -1
    else:
        return 1 if "epilepsy" in disease_name else 0

In [ ]:
LFs = [
    #LF_debugger
    LF_between_tag,
    LF_mutation,
    LF_check_disease_tag,
]

# Label The Candidates

This block of code will run through the label functions and label each candidate in the training and development groups.

In [ ]:
from snorkel.annotations import LabelAnnotator
labeler = LabelAnnotator(f=LFs)

%time L_train = labeler.apply(split=0)
%time L_dev = labeler.apply_existing(split=1)

# Genreate Coverage Stats

Before throwing our labels at a machine learning algorithm take a look at some quick stats. The code below will show the coverage of each label function and some other stat things. 

In [ ]:
print L_train.lf_stats(session, )

In [ ]:
print L_dev.lf_stats(session, )